#### 1. Observability
Monitoring:
- Input-Output prompts.
- Token usage.
- Latency.
- Cost.
- Runtime errors.

#### 2. Metrics
Numerically measure the accuracy and quality. <br>
Also helps in thinking of best case scenarios

#### 3. Optimizations
Finetuning LMs or prompts to improve metrics.

### CLI Command
`uv run mlflow server --backend-store-uri sqlite:///mydb.sqlite --port 5000`

In [3]:
import dspy
from dspy.teleprompt import mipro_optimizer_v2

from google import genai
from openai import OpenAI

import mlflow

from typing import List, Optional
from pydantic import BaseModel, Field

import time
import asyncio

import os
from dotenv import load_dotenv

load_dotenv()

True